In [1]:
# Imports
import pickle
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('../../../')

from src import customFunc as cf
from src import ansatzs as anz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.providers.fake_provider import FakeMelbourneV2
import time
from qiskit.quantum_info import Statevector

In [2]:
with open(f"data/ansatz1.000.pkl", "rb") as f:
    ansatz_naive = pickle.load(f)

backend = FakeMelbourneV2()

In [3]:
# Transpile circuit with a pass manager for a backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
t0 = time.time()
transpiled_ansatz_naive = pm.run(ansatz_naive)
t1 = time.time()
t_optimized=t1 - t0

# Show results
print(f"Transpilation time: {t_optimized} (s)")
print(f'Circuit info: {transpiled_ansatz_naive.count_ops()}')
print(f'Depth: {transpiled_ansatz_naive.depth()}')

Transpilation time: 97.93181848526001 (s)
Circuit info: OrderedDict([('cx', 14222), ('rz', 6446), ('sx', 4435), ('x', 272)])
Depth: 19377


In [ ]:
transpiled_ansatz_opt, num_cx, ansatz_opt =anz.iterate_ansatz_opt(ansatz_naive, backend)
# Show results
print(f'Circuit info: {transpiled_ansatz_opt.count_ops()}')
print(f'Depth: {transpiled_ansatz_opt.depth()}')

Circuit info: OrderedDict([('cx', 13379), ('rz', 6887), ('sx', 4533), ('x', 667)])
Depth: 18705


In [ ]:
# Transpile circuit with a pass manager for a backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
t0 = time.time()
transpiled_ansatz_opt_2 = pm.run(ansatz_opt)
t1 = time.time()
t_optimized=t1 - t0

# Show results
print(f"Transpilation time: {t_optimized} (s)")
print(f'Circuit info: {transpiled_ansatz_opt_2.count_ops()}')
print(f'Depth: {transpiled_ansatz_opt_2.depth()}')

Transpilation time: 46.21081757545471 (s)
Circuit info: OrderedDict([('cx', 13336), ('rz', 6624), ('sx', 4475), ('x', 547)])
Depth: 18609


## Comparison of ansatz

In [11]:
# Randomización de parámetros
num_params = ansatz_naive.num_parameters
x0 = np.random.uniform(0,2*np.pi,num_params)

# Ansatz sin optimizar
qc_1 = ansatz_naive.assign_parameters({list(ansatz_naive.parameters)[i]: x0[i] for i in range(len(x0))}) # Asignamos los parámetros al circuito
Sv_1 = Statevector(qc_1)
display(Sv_1.draw('latex'))

# Ansatz optimizado
qc_2 = ansatz_opt.assign_parameters({list(ansatz_opt.parameters)[i]: x0[i] for i in range(len(x0))}) # Asignamos los parámetros al circuito
Sv_2 = Statevector(qc_2)
display(Sv_2.draw('latex'))

# Fidelidad y distancia
print(abs(np.dot(Sv_1,Sv_2)))
print(abs(np.linalg.norm(Sv_1-Sv_2)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

0.9999999999987245
1.5385793906142057e-13
